In [407]:
import pandas as pd
import numpy as np

In [408]:
r = 'data2/'
flux1 = pd.read_csv(r + 'flux1.csv', sep=';', encoding='cp1252')
flux2 = pd.read_csv(r + 'flux2.csv', sep=';', encoding='cp1252')

In [409]:
flux1.head(n=3)

,Etablissement SSR,Catégorie,Code,Groupe planification,Provenance,Effectif
0,CH AUTUN SITE LATOUCHE,CH,X08,Pneumologie,CH AUTUN SITE PARPAS,30
1,CH AUTUN SITE LATOUCHE,CH,X03,Neurologie médicale,CH AUTUN SITE PARPAS,30
2,CH AUTUN SITE LATOUCHE,CH,X07,Affections Cardio-vasculaires,CH AUTUN SITE PARPAS,16


In [410]:
flux2.head(n=3)

,Finess,Raison sociale,Catégorie,Provenance,Nb de transferts
0,710978248,CH AUTUN SITE LATOUCHE,CH,CH AUTUN SITE PARPAS,155
1,710978248,CH AUTUN SITE LATOUCHE,CH,CLINIQUE DU PARC,11
2,580780187,CLINIQUE DU MORVAN SA,Privé,CH AUTUN SITE PARPAS,48


### renommage vars

In [411]:
flux1.columns = ['etablissement', 'categorie', 'code', 'specialite', 'provenance', 'effectif_transfert']
flux2.columns = ['finess', 'etablissement', 'categorie', 'provenance', 'effectif_transfert']

### cleaning strings

In [558]:
def clean_string(s):
    s = s.str.upper()\
    .str.lstrip()\
    .str.rstrip()\
    .str.replace('\t', '')\
    .str.replace('  ', ' ')
    #get ride of accents
    s = s.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')    
    return s

In [559]:
flux1['etablissement'] = clean_string(flux1['etablissement'])
flux1['provenance'] = clean_string(flux1['provenance'])

flux2['etablissement'] = clean_string(flux2['etablissement'])
flux2['provenance'] = clean_string(flux2['provenance'])

In [560]:
coord = pd.read_csv('data/new_data/coord.csv', index_col=0)

In [563]:
coord['etablissement'] = clean_string(coord['etablissement'])

### creation table ght + verif etab rattachés aux ghts

In [566]:
hp = pd.read_csv('data/new_data/hp.csv', index_col=0)

In [567]:
tmp = hp[['nom_ght', 'etablissement']].drop_duplicates()

In [568]:
tmp['etablissement'] = clean_etab(tmp['etablissement'])

In [569]:
tmp['nom_ght'] = clean_etab(tmp['nom_ght'])
tmp.loc[tmp.nom_ght == 'GHT \xa0NORD-YONNE', 'nom_ght'] = 'GHT NORD-YONNE'

In [571]:
tmp['nom_ght'] = clean_string(tmp['nom_ght'])
tmp['etablissement'] = clean_string(tmp['etablissement'])

# 11 GHT bourgogne Franche comté


source = https://www.bourgogne-franche-comte.ars.sante.fr/onze-groupements-hospitaliers-de-territoire-en-bourgogne-franche-comte

**GHT 21-52 Côte d’Or-Haute-Marne**

Chez off mais pas chez ns :

- CH Semur-en-Auxois
- CH de la Haute Côte d’Or  
- CH Auxonne

Chez nous mais pas chez rpz officielle :
- HÔPITAL LE BOCAGE CHRU DIJON
- CH HCO SITE DE CHATILLON SUR SEINE
- CH HCO SITE DE MONTBARD
- CH HCO SITE DE SAULIEU

Ds les 2 : 
- CHU Dijon (établissement support)  OK
- CH La Chartreuse Dijon  OK
- CH Is sur Tille OK
- CH Chaumont  OK
- CH  Langres  OK
- CH Bourbonne-les-Bains  OK


**GHT de la Nièvre** 

Chez off mais pas chez ns :
- CH de l’Agglomération de Nevers (établissement support) 
- CH Henri Dunant La Charité sur Loire
- CLS de St-Pierre le Moutiers
- CLS de Luzy

Chez ns mais pas chez off :

- HÔPITAL PIERRE BEREGOVOY
- CENTRE COLBERT MEDECINE ET SSR


Ds les 2 :
- CH de Decize,  OK
- CH de Cosne-Cours-sur-Loire,   OK
- CH de Château Chinon,  OK
- CH de Lormes,  OK
- CH Pierre Lôo La Charité sur Loire  OK


**GHT de Saône-et-Loire-Bresse-Morvan**

Chez off mais pas chez ns :
- CH Montceau-les-Mines
- CH Les Marronniers Toulon-sur-Arroux
- CH La Guiche

Chez ns mais pas chez off :
- CH JEAN BOUVERI

Ds les 2 :
- CH William Morey Chalon-sur-Saône (établissement support)  OK
- CH Autun OK
- CH de la Bresse Louhannaise Louhans OK
- CH Chagny  OK
- CH Sevrey  OK



**GHT de Bourgogne méridionale**

chez off mais pas chez ns:
- CH Tournus

chez ns mais pas chez off:
- CH BELNAY

ds les 2:
- CH Mâcon (établissement support)  OK
- CH du Pays Charolais-Brionnais (fusion des CH de Paray-le-Monial, Charolles, La Clayette au 01 01 2020) OK
- CH du Clunisois (Cluny-Tramayes) OK
- CH Fondation d’Aligre Bourbon-Lancy OK

**GHT du Sud Yonne-Haut-Nivernais**

Chez off mais pas chez ns :
- CHS de l’Yonne

Chez ns mais pas chez off :
- CHS AUXERRE

Ds les 2 :
- CH Auxerre (établissement support) OK
- CH Avallon OK
- CH Clamecy OK
- CH Tonnerre OK




**GHT du Nord Yonne**

Ds off et pas chez ns :
- CH Villeneuve-sur-Yonne

Les 2:
- CH Sens (établissement support) OK
- CH Joigny  OK


**GHT Sud Côte d’Or** 

Dans off mais pas chez ns :

- EHPAD Auguste Arvier de Bligny sur Ouche
- EHPAD Jeanne Pierrette Carnot de Nolay
- EHPAD Cordelier de Labergement-les-Seurre
- EHPAD La Saône de Saint-Jean-de-Losne

Les 2 :
- Centre Hospitalier Les Hospices Civils de Beaune (établissement support) OK



### GHTs manquants 

On a 7 GHT vs eux 11. 
Les ghts manquants : 

- GHT centre Franche-Comté
- GHT du Jura
- GHT de la Haute-Saône 
- GHT nord Franche-Comté 



In [573]:
flux1_etab = pd.concat([flux1.etablissement, flux1.provenance]).drop_duplicates().sort_values()
flux2_etab = pd.concat([flux2.etablissement, flux2.provenance]).drop_duplicates().sort_values()

In [574]:
print(flux1_etab.count(), flux2_etab.count())

78 92


In [575]:
flux1_etab[flux1_etab.isin(flux2_etab) == False] #6

Series([], dtype: object)

In [422]:
#C.R.B LES ROSIERS, manque le point
#CH LES CHANAUX MACON' manque le A
#'CH REG UNIVERSITAIRE DIJ' manque le ON
# CLINIQUE MEDICO-CHIR manque CHIRURGICALE'
#SA MAISON DE JOUVENCE

### on se débarasse des discrepancies etablissement (et provenance)


In [576]:
##C.R.B LES ROSIERS, manque le point
tmp.loc[tmp.etablissement == 'C.R.B. LES ROSIERS', 'etablissement']
coord.loc[coord.etablissement == 'C.R.B. LES ROSIERS', 'etablissement']
flux1.loc[flux1.etablissement == 'C.R.B. LES ROSIERS', 'etablissement']
flux2.loc[flux2.etablissement == 'C.R.B. LES ROSIERS', 'etablissement'] = 'C.R.B LES ROSIERS'

In [577]:
#flux1.loc[flux1.provenance == 'C.R.B. LES ROSIERS', 'provenance']
flux2.loc[flux2.provenance == 'C.R.B. LES ROSIERS', 'provenance'] = 'C.R.B LES ROSIERS'

In [578]:
#CH LES CHAUNAUX MACON' => 1er U en trop 
#tmp.loc[tmp.etablissement == 'CH LES CHANAUX MACON', 'etablissement']
#coord.loc[coord.etablissement == 'CH LES CHANAUX MACON', 'etablissement']
flux1.loc[flux1.etablissement == 'CH LES CHAUNAUX MACON', 'etablissement'] = 'CH LES CHANAUX MACON'
#flux2.loc[flux2.etablissement == 'CH LES CHANAUX MACON', 'etablissement']

In [579]:
flux1.loc[flux1.provenance == 'CH LES CHAUNAUX MACON', 'provenance'] = 'CH LES CHANAUX MACON'
flux2.loc[flux2.provenance == 'CH LES CHAUNAUX MACON', 'provenance'] = 'CH LES CHANAUX MACON'

In [580]:
#tmp.loc[tmp.etablissement == 'CH REG UNIVERSITAIRE DIJ', 'etablissement']
#coord.loc[coord.etablissement == 'CH REG UNIVERSITAIRE DIJ', 'etablissement']
#flux1.loc[flux1.etablissement == 'CH REG UNIVERSITAIRE DIJ', 'etablissement']
flux2.loc[flux2.etablissement == 'CH REG UNIVERSITAIRE DIJ', 'etablissement'] = 'CH REG UNIVERSITAIRE DIJON'

In [581]:
flux1.loc[flux1.provenance == 'CH REG UNIVERSITAIRE DIJ', 'provenance'] = 'CH REG UNIVERSITAIRE DIJON'
flux2.loc[flux2.provenance == 'CH REG UNIVERSITAIRE DIJ', 'provenance'] = 'CH REG UNIVERSITAIRE DIJON'

In [582]:
# CLINIQUE MEDICO-CHIR manque CHIRURGICALE'
#tmp.loc[tmp.etablissement == 'CLINIQUE MEDICO-CHIRURGICALE', 'etablissement']
#coord.loc[coord.etablissement == 'CLINIQUE MEDICO-CHIRURGICALE', 'etablissement']
flux1.loc[flux1.etablissement == 'CLINIQUE MEDICO-CHIRURGICALE', 'etablissement']
flux2.loc[flux2.etablissement == 'CLINIQUE MEDICO-CHIR', 'etablissement'] = 'CLINIQUE MEDICO-CHIRURGICALE'

In [583]:
flux1.loc[flux1.provenance == 'CLINIQUE MEDICO-CHIR', 'provenance'] = 'CLINIQUE MEDICO-CHIRURGICALE'
flux2.loc[flux2.provenance == 'CLINIQUE MEDICO-CHIR', 'provenance'] = 'CLINIQUE MEDICO-CHIRURGICALE'

In [584]:
#CH PAYS CHAROLAIS BRIONNAIS LA CLAYETT
flux2.loc[flux2.etablissement == 'CH PAYS CHAROLAIS BRIONNAIS LA CLAYETT', 'etablissement'] = 'CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE'
flux2.loc[flux2.provenance == 'CH PAYS CHAROLAIS BRIONNAIS LA CLAYETT', 'provenance'] = 'CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE'

In [585]:
#SA MAISON DE JOUVENCE
#tmp.loc[tmp.etablissement == 'SA MAISON DE JOUVENCE', 'etablissement']
coord.loc[coord.etablissement == 'SA MAISON JOUVENCE', 'etablissement'] = 'SA MAISON DE JOUVENCE'
flux1.loc[flux1.etablissement == 'SA MAISON JOUVENCE', 'etablissement'] = 'SA MAISON DE JOUVENCE'
#flux2.loc[flux2.etablissement == 'SA MAISON DE JOUVENCE', 'etablissement']

In [587]:
r = 'data3/'
tmp.to_csv(r + 'ght.csv')
coord.to_csv(r + 'coord.csv')
flux1.to_csv(r + 'flux1.csv')
flux2.to_csv(r + 'flux2.csv')

Dans flux2 mais pas dans flux1

In [441]:
flux1_etab = pd.concat([flux1.etablissement, flux1.provenance]).drop_duplicates().sort_values()
flux2_etab = pd.concat([flux2.etablissement, flux2.provenance]).drop_duplicates().sort_values()

LISTE
- CH DE BAR SUR AUBE ? 

In [442]:
flux2.iloc[172,:]

finess                                              710978271
etablissement           CH PAYS CHAROLAIS BRIONNAIS CHAROLLES
categorie                                                  CH
provenance            CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE
effectif_transfert                                         41
Name: 172, dtype: object

In [479]:
etab_in_flux2_not_flux1 = flux2_etab[flux2_etab.isin(flux1_etab) == False] #14

In [480]:
etab_in_flux2_not_flux1

150                         CH DE BAR SUR AUBE
169                              CH DE BEAUJEU
154                  CH DE BOURBONNE-LES-BAINS
172    CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE
10                        CHRU LE BOCAGE DIJON
153                  CLINIQUE DE LA COMPASSION
173                            CR LES ARBELLES
202                   GCS PATCS CRRF PASTEUR 2
199      GCS PLAT AVAL TERR CHAMP SUD CRF PAST
94                        HL LES CYGNES LORMES
6                        HOTEL DIEU DU CREUSOT
151                       HÔPITAL DE JOINVILLE
187                      MOYEN SEJOUR PIGNELIN
147                    SARL JOUVENCE NUTRITION
dtype: object

In [444]:
flux1_etab[flux1_etab.str.find('JOU') != -1]

367     MOYEN SEJOUR CH SENS
206    SA MAISON DE JOUVENCE
dtype: object

In [438]:
coord_etab = coord['etablissement']

In [439]:
coord_etab[coord_etab.str.find('PAYS') != -1]

27             CH DU PAYS CHAROLAIS BRIONNAIS
37      CH PAYS CHAROLAIS BRIONNAIS CHAROLLES
38    CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE
Name: etablissement, dtype: object

### Merging flux1 & flux2

In [766]:
df1 = flux1.groupby(['etablissement', 'provenance'])\
    .agg(_sum = ('effectif_transfert', sum)).sort_values(by='etablissement')
df1[:4]

_sum
etablissement        provenance                         
AIHP CENTRE ARMANCON CHS AUXERRE                      30
C.R.B LES ROSIERS    HOPITAL LE BOCAGE CHRU DIJON    135
                     HOPITAL PRIVE DIJON BOURGOGNE   342
                     POLYCLINIQUE DU PARC DREVON      11

In [767]:
df2 = flux2.groupby(['etablissement', 'provenance'])\
    .agg(_sum = ('effectif_transfert', sum)).sort_values(by='etablissement')

In [768]:
df2[:4]

_sum
etablissement        provenance                         
AIHP CENTRE ARMANCON CHS AUXERRE                      30
C.R.B LES ROSIERS    CLINIQUE MEDICO-CHIRURGICALE     15
                     HOPITAL LE BOCAGE CHRU DIJON    186
                     HOPITAL PRIVE DIJON BOURGOGNE   364

In [769]:
df3 = pd.merge(df1,df2, how='inner', 
               left_on=['etablissement', 'provenance'], 
               right_on=['etablissement', 'provenance'],
                suffixes=['1', '2']        
        )

In [770]:
df3['effectif_transfert'] = df3._sum2 - df3._sum1

In [771]:
df3 = df3.loc[df3.effectif_transfert != 0, :].drop(['_sum1', '_sum2'], axis=1)

In [772]:
flux1.columns

Index(['etablissement', 'categorie', 'code', 'specialite', 'provenance',
       'effectif_transfert'],
      dtype='object')

In [773]:
df3['categorie'] = np.nan
df3['code'] = np.nan
df3['specialite'] = 'Autre'

In [774]:
df3 = df3.reset_index()

In [775]:
df = pd.concat([flux1, df3])

In [776]:
df

,etablissement,categorie,code,specialite,provenance,effectif_transfert
0,CH AUTUN SITE LATOUCHE,CH,X08,Pneumologie,CH AUTUN SITE PARPAS,30
1,CH AUTUN SITE LATOUCHE,CH,X03,Neurologie médicale,CH AUTUN SITE PARPAS,30
2,CH AUTUN SITE LATOUCHE,CH,X07,Affections Cardio-vasculaires,CH AUTUN SITE PARPAS,16
3,CH AUTUN SITE LATOUCHE,CH,X06,Rhumatologie,CH AUTUN SITE PARPAS,12
4,CENTRE ORTHOPEDIQUE MEDICO-CHIR,Privé,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",CH WILLIAM MOREY CHALON SUR SAONE,78
...,...,...,...,...,...,...
103,SSR EDITH CAVELL,NaN,NaN,Autre,CH REG UNIVERSITAIRE DIJON,40
104,SSR EDITH CAVELL,NaN,NaN,Autre,HOPITAL PRIVE DIJON BOURGOGNE,51
105,SSR MARGUERITE BOUCICAUT,NaN,NaN,Autre,CH WILLIAM MOREY CHALON SUR SAONE,79
106,SSR MARGUERITE BOUCICAUT,NaN,NaN,Autre,CH JEAN BOUVERI,20


In [777]:
df.groupby(['etablissement', 'provenance', 'specialite'])\
    .agg(_sum = ('effectif_transfert', sum)).sort_values(by='etablissement')[:20]

_sum
etablissement                  provenance                     specialite                                              
AIHP CENTRE ARMANCON           CHS AUXERRE                    Effets nocifs, alcool, toxicologie, allergies         30
C.R.B LES ROSIERS              HOPITAL LE BOCAGE CHRU DIJON   Autre                                                 51
                                                              Chir. majeure de l'app. Locomoteur:\n hanche, f...    74
                                                              Chirurgies autres de l'appareil locomoteur, \na...    26
                                                              Pneumologie                                           22
                                                              Rhumatologie                                          13
                               HOPITAL PRIVE DIJON BOURGOGNE  Arthroscopies, Biopsies ostéo-articulaires            30
                                                              Autre                                                 22
                                                              Chir. majeure de l'app. Locomoteur:\n hanche, f...   269
                                                              Chirurgie du rachis, Neuro-chirurgie                  17
                                                              Chirurgies autres de l'appareil locomoteur, amp...    26
                               POLYCLINIQUE DU PARC DREVON    Autre                                                  3
                                                              Chir. majeure de l'app. Locomoteur:\n hanche, f...    11
CENTRE COLBERT MEDECINE ET SSR HOPITAL PIERRE BEREGOVOY       Neurologie médicale                                   14
                                                              Hépato-Gastro-Entérologie                             20
                                                              Chir. majeure de l'app. Locomoteur:\n hanche, f...    23
                                                              Pneumologie                                           27
                               CENTRE COLBERT MEDECINE ET SSR Rhumatologie                                          16
                                                              Psychiatrie                                           19
                               HOPITAL PIERRE BEREGOVOY       Affections Cardio-vasculaires                         18

In [778]:
df.groupby('etablissement')['effectif_transfert'].sum()[:30]

etablissement
AIHP CENTRE ARMANCON                   30
C.R.B LES ROSIERS                     564
CENTRE COLBERT MEDECINE ET SSR        137
CENTRE DE CONVALESCENCE               392
CENTRE MEDICAL DE LA VENERIE          304
CENTRE MEDICAL LA GRANGE/MONT          27
CENTRE ORTHOPEDIQUE MEDICO-CHIR       829
CENTRE SSR DU CHALONNAIS              226
CH ALIGRE BOURBON LANCY               108
CH AUTUN SITE LATOUCHE                155
CH AUXERRE                            587
CH AVALLON                            210
CH BELNAY                             271
CH BRESSE LOUHANNAISE                  96
CH CHATEAU-CHINON                      65
CH COSNE COURS SUR LOIRE              145
CH D'AUXONNE                           72
CH D'IS-SUR-TILLE                      63
CH DE CHAUMONT                        210
CH DE LA GUICHE                       176
CH DE LANGRES                         223
CH DE PONT DE VAUX                    111
CH DECIZE                             228
CH DU CLUNISOIS     

In [779]:
flux1_aug = df

### Check du merging 

Marche pour certains couples mais ! 
probleme => pas les mêmes appelations entre les 2 tableaux

In [780]:
flux1_aug.loc[(flux1_aug.etablissement == 'C.R.B LES ROSIERS') 
             & (flux1_aug.provenance == 'HOPITAL PRIVE DIJON BOURGOGNE')
             ]

,etablissement,categorie,code,specialite,provenance,effectif_transfert
182,C.R.B LES ROSIERS,Privé,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",HOPITAL PRIVE DIJON BOURGOGNE,269
183,C.R.B LES ROSIERS,Privé,C02,"Chirurgie du rachis, Neuro-chirurgie",HOPITAL PRIVE DIJON BOURGOGNE,17
184,C.R.B LES ROSIERS,Privé,C11,"Chirurgies autres de l'appareil locomoteur, am...",HOPITAL PRIVE DIJON BOURGOGNE,26
185,C.R.B LES ROSIERS,Privé,C10,"Arthroscopies, Biopsies ostéo-articulaires",HOPITAL PRIVE DIJON BOURGOGNE,30
1,C.R.B LES ROSIERS,NaN,NaN,Autre,HOPITAL PRIVE DIJON BOURGOGNE,22


In [781]:
_df1 = flux1_aug.groupby(['etablissement', 'provenance'])['effectif_transfert'].sum().reset_index()

In [782]:
_df2 = flux2.groupby(['etablissement', 'provenance'])['effectif_transfert'].sum().reset_index()

In [783]:
compare = pd.merge(_df1, _df2, how='inner', on= ['etablissement', 'provenance'])

In [784]:
compare

,etablissement,provenance,effectif_transfert_x,effectif_transfert_y
0,AIHP CENTRE ARMANCON,CHS AUXERRE,30,30
1,C.R.B LES ROSIERS,HOPITAL LE BOCAGE CHRU DIJON,186,186
2,C.R.B LES ROSIERS,HOPITAL PRIVE DIJON BOURGOGNE,364,364
3,C.R.B LES ROSIERS,POLYCLINIQUE DU PARC DREVON,14,14
4,CENTRE DE CONVALESCENCE,HOPITAL LE BOCAGE CHRU DIJON,306,306
...,...,...,...,...
108,SSR EDITH CAVELL,HOPITAL LE BOCAGE CHRU DIJON,301,301
109,SSR EDITH CAVELL,HOPITAL PRIVE DIJON BOURGOGNE,95,95
110,SSR MARGUERITE BOUCICAUT,CH JEAN BOUVERI,31,31
111,SSR MARGUERITE BOUCICAUT,CH WILLIAM MOREY CHALON SUR SAONE,301,301


In [785]:
compare.loc[compare.effectif_transfert_x != compare.effectif_transfert_y, :]

,etablissement,provenance,effectif_transfert_x,effectif_transfert_y


### Ajouter à flux1 (augmenté) les couples qui sont uniquement ds flux2

vérifier que etab / provenance / sum transfert pour flux et flux2 sont égaux

vérifier que etab / sum transfert pour flux et flux2 sont égaux

In [786]:
c1 = flux1_aug[['etablissement', 'provenance']].drop_duplicates()
c2 = flux2[['etablissement', 'provenance']].drop_duplicates()

In [787]:
print(c1.shape[0], c2.shape[0])

115 203


In [788]:
union = pd.merge(c1, c2, how='outer', indicator=True)
#union.loc[union._merge == 'left_only', :] #17

In [789]:
#couples that are in c2 but not in C1 
x = union.loc[union._merge == 'right_only', :].drop(['_merge'], axis=1)

In [790]:
etab_in_flux2_not_flux1

150                         CH DE BAR SUR AUBE
169                              CH DE BEAUJEU
154                  CH DE BOURBONNE-LES-BAINS
172    CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE
10                        CHRU LE BOCAGE DIJON
153                  CLINIQUE DE LA COMPASSION
173                            CR LES ARBELLES
202                   GCS PATCS CRRF PASTEUR 2
199      GCS PLAT AVAL TERR CHAMP SUD CRF PAST
94                        HL LES CYGNES LORMES
6                        HOTEL DIEU DU CREUSOT
151                       HÔPITAL DE JOINVILLE
187                      MOYEN SEJOUR PIGNELIN
147                    SARL JOUVENCE NUTRITION
dtype: object

In [791]:
df = pd.merge(x, flux2, how='left', left_on=['etablissement', 'provenance'], right_on=['etablissement', 'provenance']).drop('finess', axis=1)

In [792]:
df['code'] = np.nan
df['specialite'] = np.nan

In [793]:
df.shape

(90, 6)

In [794]:
flux1.shape

(414, 6)

In [800]:
flux = pd.concat([flux1_aug, df])

In [801]:
flux

,etablissement,categorie,code,specialite,provenance,effectif_transfert
0,CH AUTUN SITE LATOUCHE,CH,X08,Pneumologie,CH AUTUN SITE PARPAS,30
1,CH AUTUN SITE LATOUCHE,CH,X03,Neurologie médicale,CH AUTUN SITE PARPAS,30
2,CH AUTUN SITE LATOUCHE,CH,X07,Affections Cardio-vasculaires,CH AUTUN SITE PARPAS,16
3,CH AUTUN SITE LATOUCHE,CH,X06,Rhumatologie,CH AUTUN SITE PARPAS,12
4,CENTRE ORTHOPEDIQUE MEDICO-CHIR,Privé,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",CH WILLIAM MOREY CHALON SUR SAONE,78
...,...,...,...,...,...,...
85,MOYEN SEJOUR CH SENS,CH,NaN,NaN,CH JOIGNY,13
86,MAISON DE REPOS STE COLOMBE,Privé,NaN,NaN,CLINIQUE PAUL PIQUET,19
87,GCS PLAT AVAL TERR CHAMP SUD CRF PAST,Privé,NaN,NaN,CH SENS,19
88,CMPR L'A.D.A.P.T. LOIRET,PSPH/EBNL,NaN,NaN,CH SENS,14


In [804]:
flux.to_csv('data3/flux.csv')

In [802]:
flux.groupby(['etablissement', 'provenance'])['effectif_transfert'].sum()

etablissement             provenance                       
AIHP CENTRE ARMANCON      CHS AUXERRE                           30
C.R.B LES ROSIERS         CLINIQUE MEDICO-CHIRURGICALE          15
                          HOPITAL LE BOCAGE CHRU DIJON         186
                          HOPITAL PRIVE DIJON BOURGOGNE        364
                          POLYCLINIQUE DU PARC DREVON           14
                                                              ... 
SSR MARGUERITE BOUCICAUT  CH LES CHANAUX MACON                  13
                          CH WILLIAM MOREY CHALON SUR SAONE    301
                          HOPITAL LE BOCAGE CHRU DIJON          77
                          HOPITAL PRIVE SAINTE MARIE            27
                          HOTEL DIEU DU CREUSOT                 13
Name: effectif_transfert, Length: 205, dtype: int64

In [803]:
flux2.groupby(['etablissement', 'provenance'])['effectif_transfert'].sum()

etablissement             provenance                       
AIHP CENTRE ARMANCON      CHS AUXERRE                           30
C.R.B LES ROSIERS         CLINIQUE MEDICO-CHIRURGICALE          15
                          HOPITAL LE BOCAGE CHRU DIJON         186
                          HOPITAL PRIVE DIJON BOURGOGNE        364
                          POLYCLINIQUE DU PARC DREVON           14
                                                              ... 
SSR MARGUERITE BOUCICAUT  CH LES CHANAUX MACON                  13
                          CH WILLIAM MOREY CHALON SUR SAONE    301
                          HOPITAL LE BOCAGE CHRU DIJON          77
                          HOPITAL PRIVE SAINTE MARIE            27
                          HOTEL DIEU DU CREUSOT                 13
Name: effectif_transfert, Length: 203, dtype: int64

In [498]:
flux2.loc[(flux2.etablissement == 'C.R.B LES ROSIERS') & (flux2.provenance == 'HOPITAL PRIVE DIJON BOURGOGNE'), :]

,finess,etablissement,categorie,provenance,effectif_transfert
136,210780292,C.R.B LES ROSIERS,Privé,HOPITAL PRIVE DIJON BOURGOGNE,364


In [499]:
flux1.groupby(['etablissement', 'provenance'])['effectif_transfert'].sum()

etablissement                   provenance                       
AIHP CENTRE ARMANCON            CHS AUXERRE                           30
C.R.B LES ROSIERS               HOPITAL PRIVE DIJON BOURGOGNE        342
                                HÔPITAL LE BOCAGE CHRU DIJON         135
                                POLYCLINIQUE DU PARC DREVON           11
CENTRE COLBERT MEDECINE ET SSR  CENTRE COLBERT MEDECINE ET SSR        35
                                                                    ... 
SSR EDITH CAVELL                HOPITAL PRIVE DIJON BOURGOGNE         44
                                HÔPITAL LE BOCAGE CHRU DIJON         199
SSR MARGUERITE BOUCICAUT        CH JEAN BOUVERI                       11
                                CH WILLIAM MOREY CHALON SUR SAONE    222
                                HÔPITAL LE BOCAGE CHRU DIJON          39
Name: effectif_transfert, Length: 115, dtype: int64

In [517]:
flux1_aug.groupby(['etablissement', 'provenance'])['effectif_transfert'].sum()

etablissement                   provenance                       
AIHP CENTRE ARMANCON            CHS AUXERRE                           30
C.R.B LES ROSIERS               HOPITAL PRIVE DIJON BOURGOGNE        364
                                HÔPITAL LE BOCAGE CHRU DIJON         135
                                POLYCLINIQUE DU PARC DREVON           14
CENTRE COLBERT MEDECINE ET SSR  CENTRE COLBERT MEDECINE ET SSR        35
                                                                    ... 
SSR EDITH CAVELL                HOPITAL PRIVE DIJON BOURGOGNE         95
                                HÔPITAL LE BOCAGE CHRU DIJON         199
SSR MARGUERITE BOUCICAUT        CH JEAN BOUVERI                       31
                                CH WILLIAM MOREY CHALON SUR SAONE    301
                                HÔPITAL LE BOCAGE CHRU DIJON          39
Name: effectif_transfert, Length: 115, dtype: int64

In [522]:
coord_etab[coord_etab.str.find('BOC') !=-1]

62    HÔPITAL LE BOCAGE CHRU DIJON
Name: etablissement, dtype: object

### vérification concordance coordonées

N'apparait pas dans flux2 => 'HOPITAL PRIVE SAINTE MARIE'

In [614]:
np.sum(~flux1_etab.isin(flux2_etab))

0

In [725]:
coord_etab = coord['etablissement'].drop_duplicates()
flux2_etab = pd.concat([flux2.etablissement, flux2.provenance]).drop_duplicates()
flux1_etab = pd.concat([flux1.etablissement, flux1.provenance]).drop_duplicates()

In [726]:
coord_etab.name = 'etablissement'
flux2_etab.name = 'etablissement'
flux1_etab.name = 'etablissement'

In [727]:
#
union = pd.merge(coord_etab, flux2_etab, how='outer', indicator=True)

In [728]:
#in coord but not in flux2
union.loc[union._merge=='left_only', :]

,etablissement,_merge
32,CH HENRI DUNANT LA CHARITE-SUR-LOIRE,left_only
45,CHS SEVREY,left_only
59,HL CHAGNY,left_only


In [673]:
tmp.loc[tmp.etablissement == 'CH DU TONNEROIS', 'etablissement'] = 'CH DU TONNERROIS'
coord.loc[coord.etablissement == 'CH DU TONNEROIS', 'etablissement'] = 'CH DU TONNERROIS'
flux1.loc[flux1.etablissement == 'CH DU TONNEROIS', 'etablissement'] = 'CH DU TONNERROIS'
flux1.loc[flux1.provenance == 'CH DU TONNEROIS', 'provenance'] = 'CH DU TONNERROIS'
flux2.loc[flux2.etablissement == 'CH DU TONNEROIS', 'etablissement'] = 'CH DU TONNERROIS'
flux2.loc[flux2.provenance == 'CH DU TONNEROIS', 'provenance'] = 'CH DU TONNERROIS'

In [752]:
def change_name(a, b):
    tmp.loc[tmp.etablissement == a, 'etablissement'] = b
    all_coord.loc[coord.etablissement == a, 'etablissement'] = b
    flux1.loc[flux1.etablissement == a, 'etablissement'] = b
    flux1.loc[flux1.provenance == a, 'provenance'] = b
    flux2.loc[flux2.etablissement == a, 'etablissement'] = b
    flux2.loc[flux2.provenance == a, 'provenance'] = b

In [753]:
change_name('CH DU TONNEROIS', 'CH DU TONNERROIS')
change_name('SARL JOUVENCE NUTRITION', 'SA MAISON DE JOUVENCE')
change_name('CHRU LE BOCAGE DIJON', 'HOPITAL LE BOCAGE CHRU DIJON')
change_name('HOTEL-DIEU DU CREUSOT', 'HOTEL DIEU DU CREUSOT')

TypeError: unhashable type: 'Series'

In [691]:
flux2_etab[flux2_etab.str.find('CHAGNY') != -1]

Series([], Name: etablissement, dtype: object)

In [697]:
#in flux2 but not in coord
union.loc[union._merge=='right_only', :].drop('_merge', axis=1)

,etablissement
78,CH DE BAR SUR AUBE
79,HOPITAL DE JOINVILLE
80,CLINIQUE DE LA COMPASSION
81,CH DE BOURBONNE-LES-BAINS
82,HOTEL DIEU DU CREUSOT
83,CLINIQUE DU PARC
84,HOPITAL PRIVE DIJON BOURGOGNE
85,CENTRE GEORGE-FRANCOIS LECLERC
86,CH LA CHARTREUSE
87,POLYCLINIQUE DU VAL DE LOIRE


In [ ]:
# CH DU TONNEROIS => CH DU TONNERROIS
# SARL JOUVENCE NUTRITION => SA MAISON DE JOUVENCE
# CHRU LE BOCAGE DIJON = >HOPITAL LE BOCAGE CHRU DIJON

In [711]:
missing_coord = pd.read_csv('data3/missing_coord.csv', sep=';', encoding='cp1252', index_col=0)
missing_coord.columns = ['etablissement', 'adresse', 'latitude', 'longitude']

missing_coord['latitude'] = missing_coord['latitude'].str.replace(',', '.')
missing_coord['longitude'] = missing_coord['longitude'].str.replace(',', '.')

In [722]:
missing_coord.loc[missing_coord.etablissement == 'CH ROBERT MORLEVAT', 'latitude'] = 47.495749
missing_coord.loc[missing_coord.etablissement == 'CH ROBERT MORLEVAT', 'longitude'] = 4.34625

missing_coord.loc[missing_coord.etablissement == 'POLYCLINIQUE DU PARC DREVON', 'latitude'] = 47.31217
missing_coord.loc[missing_coord.etablissement == 'POLYCLINIQUE DU PARC DREVON', 'longitude'] = 5.04348

missing_coord.loc[missing_coord.etablissement == 'CLINIQUE MEDICO-CHIRURGICALE', 'latitude'] = 47.36096
missing_coord.loc[missing_coord.etablissement == 'CLINIQUE MEDICO-CHIRURGICALE', 'longitude'] =  5.05105

In [723]:
all_coord = pd.concat([coord, missing_coord])

In [724]:
all_coord

,etablissement,adresse,latitude,longitude
0,AIHP CENTRE ARMANCON,18 Bis Rue Pierre Semard,47.95981,3.53041
1,C.R.B LES ROSIERS,45 Bd Henri Bazin,47.30041,5.01900
2,CENTRE COLBERT MEDECINE ET SSR,4 Rue Étienne Litaud,46.99469,3.15970
3,CENTRE DE CONVALESCENCE,67 Rte d'Ahuy,47.34583,5.03471
4,CENTRE MEDICAL DE LA VENERIE,Lieu Dit La Vénerie,47.2668,3.28960
...,...,...,...,...
88,POLYCLINIQUE DU VAL DE LOIRE,49 Bd Jérôme Trésaguet,46.99564,3.14828
89,CLINIQUE PAUL PIQUET,12 Rue Pierre Castets,48.20712,3.29428
90,CH ROBERT MORLEVAT,3 Av. Pasteur,47.495749,4.34625
91,POLYCLINIQUE DU PARC DREVON,18 Cr Général de Gaulle,47.31217,5.04348


In [731]:
#last check
union.loc[union._merge=='right_only', :]

,etablissement,_merge
78,CH DE BAR SUR AUBE,right_only
79,HOPITAL DE JOINVILLE,right_only
80,CLINIQUE DE LA COMPASSION,right_only
81,CH DE BOURBONNE-LES-BAINS,right_only
82,HOTEL DIEU DU CREUSOT,right_only
83,CLINIQUE DU PARC,right_only
84,HOPITAL PRIVE DIJON BOURGOGNE,right_only
85,CENTRE GEORGE-FRANCOIS LECLERC,right_only
86,CH LA CHARTREUSE,right_only
87,POLYCLINIQUE DU VAL DE LOIRE,right_only


In [760]:
all_coord.loc[all_coord.etablissement== 'HOTEL-DIEU DU CREUSOT', 'etablissement'] = 'HOTEL DIEU DU CREUSOT'

In [761]:


coord_etab = all_coord['etablissement'].drop_duplicates()
flux2_etab = pd.concat([flux2.etablissement, flux2.provenance]).drop_duplicates()
flux1_etab = pd.concat([flux1.etablissement, flux1.provenance]).drop_duplicates()

coord_etab.name = 'etablissement'
flux2_etab.name = 'etablissement'
flux1_etab.name = 'etablissement'

union = pd.merge(coord_etab, flux2_etab, how='outer', indicator=True)

In [762]:
union.loc[union._merge=='right_only']

,etablissement,_merge


In [764]:
all_coord

,etablissement,adresse,latitude,longitude
0,AIHP CENTRE ARMANCON,18 Bis Rue Pierre Semard,47.95981,3.53041
1,C.R.B LES ROSIERS,45 Bd Henri Bazin,47.30041,5.01900
2,CENTRE COLBERT MEDECINE ET SSR,4 Rue Étienne Litaud,46.99469,3.15970
3,CENTRE DE CONVALESCENCE,67 Rte d'Ahuy,47.34583,5.03471
4,CENTRE MEDICAL DE LA VENERIE,Lieu Dit La Vénerie,47.2668,3.28960
...,...,...,...,...
88,POLYCLINIQUE DU VAL DE LOIRE,49 Bd Jérôme Trésaguet,46.99564,3.14828
89,CLINIQUE PAUL PIQUET,12 Rue Pierre Castets,48.20712,3.29428
90,CH ROBERT MORLEVAT,3 Av. Pasteur,47.495749,4.34625
91,POLYCLINIQUE DU PARC DREVON,18 Cr Général de Gaulle,47.31217,5.04348


In [765]:
flux1_aug

,etablissement,categorie,code,specialite,provenance,effectif_transfert
0,CH AUTUN SITE LATOUCHE,CH,X08,Pneumologie,CH AUTUN SITE PARPAS,30
1,CH AUTUN SITE LATOUCHE,CH,X03,Neurologie médicale,CH AUTUN SITE PARPAS,30
2,CH AUTUN SITE LATOUCHE,CH,X07,Affections Cardio-vasculaires,CH AUTUN SITE PARPAS,16
3,CH AUTUN SITE LATOUCHE,CH,X06,Rhumatologie,CH AUTUN SITE PARPAS,12
4,CENTRE ORTHOPEDIQUE MEDICO-CHIR,Privé,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",CH WILLIAM MOREY CHALON SUR SAONE,78
...,...,...,...,...,...,...
92,SSR EDITH CAVELL,NaN,NaN,Autre,CLINIQUE MEDICO-CHIRURGICALE,70
93,SSR EDITH CAVELL,NaN,NaN,Autre,CH REG UNIVERSITAIRE DIJON,40
94,SSR EDITH CAVELL,NaN,NaN,Autre,HOPITAL PRIVE DIJON BOURGOGNE,51
95,SSR MARGUERITE BOUCICAUT,NaN,NaN,Autre,CH WILLIAM MOREY CHALON SUR SAONE,79


In [809]:
#LAST SAVE
r = 'data4/'
flux.to_csv(r + 'flux.csv')
all_coord.to_csv(r + 'all_coord.csv')
tmp.to_csv(r + 'ght.csv')
flux1.to_csv(r + 'flux1.csv')
flux2.to_csv(r + 'flux2.csv')
flux1_aug.to_csv(r + 'flux1_aug.csv')